# Исследование надежности заемщиков.

# Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(15)
# data.tail(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Описание данных**

* `children` - количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

In [3]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [4]:
data['children'].max() 

20

* Возможно ли такое кол-во детей или нет, мы не знаем. Может быть это человеческий фактор при заполнении сказался. Так как у нас стоит вопрос: `"Есть ли зависимость между количеством детей и возвратом кредита в срок?"` это может повлить. 

In [5]:
data['days_employed'].mean() / 365 # среднее поделил на кол-во дней в году и получил среднюю продолжительность стажа

172.73013057937914

* Такое невозможно, ну только если ты не вампир. Видимо, ошибка при заполнении данных (но это не точно). 

In [6]:
data['dob_years'].min()

0

* `dob_years` — возраст клиента в годах. Такого рода ошибка не понятна. Почему произошла тоже неизвестно. Может быть не указали, но как тогда кредит выдали. Может быть человечский фактор.

**Обнаруженные ошибки**

* Видны пропущенные значения в столбцах: `days_employed, total_income`. В столбце `days_employed` присутствуют отрицательные значения. Также в этой колонке есть слишком большие стажи (такое мало вероятно в наших реалиях). Неизвестно по какой причине и нужны ли нам эти значения вообще;
* В колонке `education`  есть прописные буквы;
* Человеческий фактор при заполнении данных или что-то пошло не так (в таких как `children` , `dob_years`)

### Шаг 2.1 Заполнение пропусков

In [7]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Обзор**

Тут мы видим количество пропущенных значений в двух столбцах: `data[['days_employed', 'total_income']]`. Мы их видели еще ранее, при помощи метода `info()`, но тут мы видим их количество

In [8]:
data.loc[(data['days_employed'].isna())&(data['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


* Тут мы видим одинаковые пропущенные значения для одних и те же строк для колонок: `days_employed и total_income`; 
* Также они имеют значение `NaN`
* Возможная причина появления: Дублирование из-за введение данных либо в доходы, либо в стаже

In [9]:
proportion_of_missing_values1 = data['days_employed'].isna().sum() / len(data) 
print(f'Доля пропущенных значений в столбце \'days_employed\' составляет: {proportion_of_missing_values1:.0%}')

Доля пропущенных значений в столбце 'days_employed' составляет: 10%


In [10]:
proportion_of_missing_values2 = data['total_income'].isna().sum() / len(data)
print(f'Доля пропущенных значений в столбце \'total_income\' составляет: {proportion_of_missing_values2:.0%}')

Доля пропущенных значений в столбце 'total_income' составляет: 10%


* Исходя из ранее сказанного, тут логично заявить, что доля пропущенных значений для двух столбцов составляет `10%`

In [11]:
data['total_income'].median()

145017.93753253992

In [12]:
data['total_income'] = data['total_income'].fillna(value=data['total_income'].median())
print(data['total_income'])

0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21525, dtype: float64


In [13]:
data.loc[55, 'total_income'] 

145017.93753253992

* Заменили пропущенные данные медианным значением по столбцу
* Проверили одну ячейку, точно ли заменились значение NaN на медиану 
* `Заполнить пропуски медианным значением — лучшее решение для количественных переменных, так как большой разброс в доходах(лучше, чем, например, считать среднее арифметическое)` Наглядный пример: "У Коли 10 яблок, у Андрея 0, в среднем у каждого по 5"

In [14]:
# data.head(15)
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Проверили еще раз на пропуски, так мы убедимся, что точно идет 'как по маслу'. Видим, что в слобце `'total_income'` пропущенные значения заменились 
                  

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [15]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Видим аномалию в виде отрицательных чисел. На что это может влиять? Проверим какое кол-во с отрицательным показателем

In [16]:
# dk = data.loc[data['days_employed'] < 0].shape

print('Количество отрицательных строк в общем трудовом стаже \'days_employed\':', data.loc[data['days_employed'] < 0]\
      .count()[0])
print('Количество положительных строк в общем трудовом стаже \'days_employed\':', data.loc[data['days_employed'] > 0]\
      .count()[0])

Количество отрицательных строк в общем трудовом стаже 'days_employed': 15906
Количество положительных строк в общем трудовом стаже 'days_employed': 3445


In [17]:
median_s_minusom = data['days_employed'].median()
print(f'Медиана со всеими значениями (отрицательными и положительными) колонки \'days_employed\': {median_s_minusom}')
mean = data['days_employed'].mean()
print(f'Среднее со всеими значениями колонки \'days_employed\': {mean}')

Медиана со всеими значениями (отрицательными и положительными) колонки 'days_employed': -1203.369528770489
Среднее со всеими значениями колонки 'days_employed': 63046.49766147338


In [18]:
data['days_employed'] = abs(data['days_employed'])
new_median_s_abs = data['days_employed'].median()
print(f'Медиана с превращенными отрицательными значениями в модульные: {new_median_s_abs}')

Медиана с превращенными отрицательными значениями в модульные: 2194.220566878695


**ВЫВОД**

* Предположу, что это просто некорректный внос данных, хотя очень большое кол-во с ошибками. Но исходя из данных и информации, что у нас есть, мы не можем знать причину возникновения данной аномалии, поэтому данные выбросы можно оставить как есть. 

In [19]:
data['children'].max() 

20

In [20]:
data['children'].value_counts().to_frame() # спасибо за этот метод из прошлого проетка

,children
0,14149
1,4818
2,2055
3,330
20,76
-1,47
4,41
5,9


Это кажется странным, хотя вполне возможно (это про 20 детей).

In [21]:
children_amount = data[data['children'] == 20].count()[0]
print(f'Количество детей в семье с 20 детьми: {children_amount}')

Количество детей в семье с 20 детьми: 76


Слишком много. Возможно, совершена опечатка.

* Предположим, что это опечатка при заполнении, когда хотели написать 0 ввели перед уже написанным нулем 2, или при наличии 2 детей случайно приписали 0. Поэтому возьмем также медианное значение.
* `Это все очень важно учитывать, так как у нас зависит от этого исследование`

In [22]:
median_children = data.loc[data['children'] != 20, 'children'].median()
# print(median_children)

In [23]:
data['children'] = data['children'].replace(20, median_children)
data['children'].value_counts().to_frame()

,children
0.0,14225
1.0,4818
2.0,2055
3.0,330
-1.0,47
4.0,41
5.0,9


In [24]:
data['children'].min()

-1.0

Странно. Возможно также опечатка (не думаю, что они имеют в виду, что когда-то был)

In [25]:
print('Количество семей с отрицательным показателем детей:', data[data['children'] == -1].count()[0])

Количество семей с отрицательным показателем детей: 47


Тут явная опечатка, по моему соображению. Наверно, имелось в виду просто `1`. Исправляем на `1`

In [26]:
data['children'] = data['children'].replace(-1,1)
# data['children'].min()
data['children'].value_counts().to_frame()

,children
0.0,14225
1.0,4865
2.0,2055
3.0,330
4.0,41
5.0,9


Видим, что после замен значения преобразовались в вещественные числа, что не очень удобно при чтении таблицы
* пользуемся методом преобразования `astype('type')`

In [27]:
data['children'] = data['children'].astype('int')
data['children'].value_counts().to_frame()

,children
0,14225
1,4865
2,2055
3,330
4,41
5,9


Так мы исправили  аномалию в колонке `children`

In [28]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.472660,66914.728907,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,0.750616,139030.880527,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [29]:
data['dob_years'].min()

0

Как было ранее сказано: `dob_years` — возраст клиента в годах. Такого рода ошибка не понятна. Почему произошла тоже неизвестно. Может быть не указали, но как тогда кредит выдали. Может быть человечский фактор. Не совсем понимаю, что делать с данными.

In [30]:
data['days_employed'].median()

2194.220566878695

In [31]:
data['days_employed'] = data['days_employed'].fillna(value=data['days_employed'].median())

* Заполнили пропуски в колонке `days_employed` медианным значением c модульными значениями

In [32]:
data['days_employed'].isna().sum()

0

* Сделали проверку проделанного

### Шаг 2.3. Изменение типов данных.

In [33]:
data['total_income'] = data['total_income'].astype('int')
# data.head(15)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


* Заменили вещественные значения в колонке `total_income` на целочисленные
* Также для удобства прочтения таблицы / для хорошего визуального вида  делаем столбец `'days_employed'` тоже целым

In [34]:
data['days_employed'] = data['days_employed'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

In [35]:
data.duplicated().sum() # При новом запуске кода обнаруживается 71, как по таблице. Тут почему-то 54

54

In [36]:
data['education'] = data['education'].str.lower()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


* В колонке `education` были прописные буквы. Привели их к одному регистру. 

Попытаемся проследить, где еще могут быть дубликаты ручным способом

In [37]:
data[data.duplicated()].sort_values(['family_status','days_employed', 'total_income', 'purpose'], ascending=False)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
11791,0,2194,47,среднее,1,женат / замужем,0,F,сотрудник,0,145017,строительство недвижимости
6312,0,2194,30,среднее,1,женат / замужем,0,M,сотрудник,0,145017,строительство жилой недвижимости
13025,1,2194,44,среднее,1,женат / замужем,0,F,сотрудник,0,145017,сделка с подержанным автомобилем
19946,0,2194,57,среднее,1,женат / замужем,0,F,сотрудник,0,145017,сделка с подержанным автомобилем
19184,0,2194,46,среднее,1,женат / замужем,0,F,сотрудник,0,145017,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
20662,0,2194,58,среднее,1,гражданский брак,1,M,сотрудник,0,145017,на проведение свадьбы
9528,0,2194,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью
19321,0,2194,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
14832,0,2194,50,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,недвижимость


* Сортируем по логической индексации дубликаты в предполагаемых столбцах;
* Обнаружили, что в колонке `family_status` прописные буквы сочетаются со строчными;
* Есть дубликаты в колонке `days_employed` и `total_income`

In [38]:
data['family_status'] = data['family_status'].str.lower()

Исправляем первую проблему

In [39]:
data[data.duplicated()].sort_values(['family_status','days_employed', 'total_income', 'purpose'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
9528,0,2194,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью
7808,0,2194,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
7921,0,2194,64,высшее,0,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
7938,0,2194,71,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
9604,0,2194,71,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
6312,0,2194,30,среднее,1,женат / замужем,0,M,сотрудник,0,145017,строительство жилой недвижимости
11791,0,2194,47,среднее,1,женат / замужем,0,F,сотрудник,0,145017,строительство недвижимости
8583,0,2194,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,145017,дополнительное образование
14832,0,2194,50,среднее,1,не женат / не замужем,4,F,сотрудник,0,145017,недвижимость


* Видим дубликаты. Предполагаемая причина, что просто произошло задвоение данных/ошибочное создание дубликатов.


In [40]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

* Тут мы видим, что некоторые параметры схожи по значению. Такие как: `'сыграть свадьбу'`, `'на проведение свадьбы'`, `'покупка жилой недвижимости'`,`'недвижимость'`, `'строительство жилой недвижимости'` и т.д.

In [41]:
data['purpose'].value_counts().to_frame()

,purpose
свадьба,797
на проведение свадьбы,777
сыграть свадьбу,774
операции с недвижимостью,676
покупка коммерческой недвижимости,664
операции с жильем,653
покупка жилья для сдачи,653
операции с коммерческой недвижимостью,651
жилье,647
покупка жилья,647


* Тут более наглядно видно, что многие строки совпадают по ключевым значениям

In [42]:
data = data.drop_duplicates().reset_index(drop=True)

* Удаляем дубликаты

In [43]:
data.duplicated().sum()

0

Проверяем еще раз проделанную работу и видим, что строки-дубликаты удалены

In [44]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [45]:
data['gender'].value_counts().to_frame()

,gender
F,14174
M,7279
XNA,1


In [46]:
data.loc[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10684,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


* Страная записаь `'XNA'`. Возможно случайная запись. Нам онa не понадобится, поэтому просто оставим

In [47]:
# data['income_type'].value_counts()

In [48]:
# data['education'].value_counts()

In [49]:
# data['education_id'].value_counts()

In [50]:
# data['family_status'].value_counts()

In [51]:
# data['debt'].value_counts()

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [52]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Создаем два новых датафрейма, в которых:
* каждому уникальному значению из `education` соответствует уникальное значение `education_id`
* каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id`

In [53]:
data['education_id'].unique()

array([0, 1, 2, 3, 4])

In [54]:
data_dict_education = data[['education_id', 'education']]
dde = data_dict_education.drop_duplicates().reset_index(drop=True).sort_values(by='education_id')
# data_dict_education.count().to_frame()
# data_dict_education.sort_values(by='education_id')
display(dde)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [55]:
data_education = pd.DataFrame(data=data, columns=['education_id', 'education'])
data_education.drop_duplicates().reset_index(drop=True).sort_values(by='education_id')
#data_education.groupby('education_id')['education']

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


* Также создаем датафрейм для `family_status` и `family_status_id` (2 способами):

In [56]:
data_family = pd.DataFrame(data=data, columns=['family_status_id', 'family_status'])
data_family.drop_duplicates().reset_index(drop=True).sort_values(by='family_status_id')

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [57]:
data_dict_family = data[['family_status_id', 'family_status']]
#data['family_status_id'].unique()
ddf = data_dict_family.drop_duplicates().reset_index(drop=True).sort_values(by='family_status_id')
display(ddf)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Удаляем из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`
* пользуемся для этого методом `drop(column=''`)

In [58]:
data['education'] = data_dict_education
# data.head(15)
data=data.drop(columns='education')
data['family_status'] = data_dict_family
data=data.drop(columns='family_status')
data.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создаем столбец `total_income_category` с категориями:

In [59]:
def total_income_category(income):
    
    
    if income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    else:
        return 'A'
    
        
# print(total_income_category(235000))
data['total_income_category'] = data['total_income'].apply(total_income_category)
data.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


Для этого мы применили функцию и указали соотвествующие диапозоны для категорий, прибегая к методу `apply()`

### Шаг 2.7. Категоризация целей кредита.

Создаем функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в котором будут категории:
* `'операции с автомобилем'`,
* `'операции с недвижимостью'`,
* `'проведение свадьбы'`,
* `'получение образования'`.

Тут мы воспользуемся ключевыми буквами значений в колонке `'purpose'`, а также используем метод `apply()`

In [60]:
def purpose_category(purpose):
    
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose:
        return 'операции с недвижимостью'
    
    elif 'свадь' in purpose:
        return 'проведение свадьбы'
    elif 'образо' in purpose:
        return 'получение образования'
    return 'операции с недвижимостью' # так как проверили на уникальные значения, там оказались все операции с жильем
    
    
        
    
data['purpose_category'] = data['purpose'].apply(purpose_category)
#data['purpose_category'].value_counts()
# data['purpose'].unique() 
data.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


**Вывод**

Так мы категоризировали данные для удобства при просмотре и чтении данных, создав две новых колонки. Объединили избранные данные в произвольные группы по заданному критерию, прибегая к функциям и методу `apply()`

### Ответы на вопросы.

##### Вопрос 1:

* Есть ли зависимость между количеством детей и возвратом кредита в срок?

Отвечая на этот вопрос, применим сводную таблицу, прибегая к методу `pivot_table()`. Для этого сгруппируем колонку `'children'` и добавим функцию подсчета зодолженности в зависимости от кол-во детей. Таблицу сгруппируем по детям и значением задолженности с функцией средней арифметической

Для удобства переименовываем колонки и добавляем функцию с процентами с подсчетом задолженности для визуального восприятия и быстрой обработки информации:

In [61]:
#data.head()
#data['children'].value_counts()
#data['debt'].unique()
#data['debt'].value_counts()
#data['debt'].sum()
#data.groupby('children').agg({'debt': ['count', 'mean']})
#children_debt = data.groupby('children').agg({'debt':'mean'})
#children_debt = data.groupby('children')['debt'].mean()
#print(f'Задолженность {children_debt:.0%}')

children_debt = data.pivot_table(index=['children'], values='debt', aggfunc=('mean'))


def my_func(i):
    return f'{i:.0%}'

children_debt['mean'] = children_debt['debt'].apply(my_func)
children_debt['count'] = data.groupby('children').agg({'debt': 'sum'}) # вспомним метод agg(), хоть здесь и одна операция
children_debt['Количество всех c таким кол-во детьми'] = data.groupby('children')['debt'].count()
children_debt = children_debt.rename(columns={'mean': 'Задержка в %', 'debt': 'Задолженность(средняя)', 'count': 'Количество  задолженностей'})
display(children_debt)


,Задолженность(средняя),Задержка в %,Количество задолженностей,Количество всех c таким кол-во детьми
children,,,,
0,0.075598,8%,1071,14167
1,0.091658,9%,445,4855
2,0.094542,9%,194,2052
3,0.081818,8%,27,330
4,0.097561,10%,4,41
5,0.000000,0%,0,9


**Вывод по вопросу №1**

Задолженности коррелируют с кол-во детьми. Зависимость между количеством детей и возвратом кредита в срок есть! Мы наблюдаем, что клиенты банка без детей ответственнее подходят к делу и отдают чаще других кредиты в срок. Но мы видим интересный факт, что при наличии у семьи `3 детей` процент задержки задолженности в среднем почти такой же, как  `без детей`.

##### Вопрос 2

* Есть ли зависимость между семейным положением и возвратом кредита в срок?

Вспомним, что мы удалили колонки `family_status` оставив их идентификаторы. Для наглядности снова применим:

In [62]:
display(ddf)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Видим, какие идентификаторы соотвествуют семейному статусу.

Создаем сводную таблицу по этим данным (аналогия решения такая же):

In [63]:
#data['family_status_id'].count()
#fs_debt
#data.groupby('family_status_id')['debt'].mean()

fs_debt = data.pivot_table(index=['family_status_id'], values='debt', aggfunc=('mean'))

def my_func1(i):
    return f'{i:.0%}'

fs_debt['Задолженность %'] = fs_debt['debt'].apply(my_func1)
fs_debt['Количество задолженностей'] = data.groupby('family_status_id')['debt'].sum()
fs_debt['Количество всех с таким статусом'] = data.groupby('family_status_id').agg({'debt': 'count'})
display(fs_debt)
ddf

,debt,Задолженность %,Количество задолженностей,Количество всех с таким статусом
family_status_id,,,,
0,0.075452,8%,931,12339
1,0.093471,9%,388,4151
2,0.065693,7%,63,959
3,0.071130,7%,85,1195
4,0.097509,10%,274,2810


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


**Вывод по вопросу №2**

Мы видим, что меньше всего задолженностей у идентификатора под номером 2 (вдова/вдовец). После идет `id` 3 (в разводе). Большой процент у `id` 1 (гражданский брак) и `id` 4 (не женат / не замужем). Тут наблюдается прямая зависимость семейного положения. Хоть и кол-во людей в разделе вдова/вдовец небольшое. Исходя из данных, можно допустить, что наличие брака может быть положительно оценен при выдаче кредита.

##### Вопрос 3:

* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Применяем такую же аналогию решения:

In [64]:
#data['debt'].value_counts()
income_debt = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=('mean'))

def my_func2(i):
    return f'{i:.0%}'

income_debt['Задолженность %'] = income_debt['debt'].apply(my_func1)
income_debt['Количество задолженностей'] = data.groupby('total_income_category')['debt'].sum()
income_debt['Количество всех с таким доходом'] = data.groupby('total_income_category').agg({'debt': 'count'})
income_debt

,debt,Задолженность %,Количество задолженностей,Количество всех с таким доходом
total_income_category,,,,
A,0.080000,8%,2,25
B,0.070621,7%,356,5041
C,0.084915,8%,1360,16016
D,0.060000,6%,21,350
E,0.090909,9%,2,22


Напомню, что мы категоризировали данные по доходами и разместили их под такими значениями:

* 0–30000 — `'E'`;
* 30001–50000 — `'D'`;
* 50001–200000 — `'C'`;
* 200001–1000000 — `'B'`;
* 1000001 и выше — `'A'`

**Вывод по вопросу №3**

Самый большой процент у людей с категорией `E` - малым доходом. Но задолженностей всего несколько, поэтому на этом выводы сложно делать. По кол-во задолженностей и проценту видно, что категория `C` большее среднее по задержкам. У категории `B` тоже есть задолженности по соотношению всех людей, которыю получают такой доход, но он по соотношению не велик. Исходя из таблицы, можно сказать, что зависимость между уровнем дохода и возвратом кредита в срок есть.

##### Вопрос 4:

* Как разные цели кредита влияют на его возврат в срок?

Снова воспользуемся сводными таблицами / методом `pivot_table()`:

In [65]:
purpose_debt = data.pivot_table(index=['purpose_category'], values='debt', aggfunc='mean')

def my_func8(i):
    return f'{i:.0%}'

purpose_debt['Задолженность %'] = purpose_debt['debt'].apply(my_func8)
purpose_debt['Количество задолженностей'] = data.groupby('purpose_category')['debt'].sum()
purpose_debt['Количество всех с такой операцией'] = data.groupby('purpose_category').agg({'debt': 'count'})
purpose_debt

,debt,Задолженность %,Количество задолженностей,Количество всех с такой операцией
purpose_category,,,,
операции с автомобилем,0.093590,9%,403,4306
операции с недвижимостью,0.072334,7%,782,10811
получение образования,0.092200,9%,370,4013
проведение свадьбы,0.080034,8%,186,2324


**Вывод по вопросу №4**

Наблюдаем, что меньше всего задолженностей у людей с операциями с недвижимостью. Предположим, что люди более ответственнее подходят к крупной покупки, либо уходом за ней. Что касается операций с автомобилем, думаю, что тут связано с крупными тратами с их эксплуатацией.

In [66]:
#data.tail(15)

## Общий вывод:

Для полноценного вывода хотелось бы посмотреть, как зависит источник дохода и цель займа от сдачи кредита в срок. Для этого снова применим метод `pivot_table()`:

In [67]:
final_debt = data.pivot_table(index=['income_type', 'purpose_category'], values='debt', aggfunc='mean')

def my_func4(i):
    return f'{i:.0%}'
    
final_debt['Сопоставленная задолженность в %'] = final_debt['debt'].apply(my_func4)
display(final_debt)
#data.loc[(data['income_type'] == 'безработный')].value_counts().to_frame()

debt  \
income_type     purpose_category                     
безработный     операции с недвижимостью  0.500000   
в декрете       операции с автомобилем    1.000000   
госслужащий     операции с автомобилем    0.076923   
                операции с недвижимостью  0.047745   
                получение образования     0.081395   
                проведение свадьбы        0.044025   
компаньон       операции с автомобилем    0.080798   
                операции с недвижимостью  0.065567   
                получение образования     0.075472   
                проведение свадьбы        0.099048   
пенсионер       операции с автомобилем    0.064151   
                операции с недвижимостью  0.048652   
                получение образования     0.066574   
                проведение свадьбы        0.059242   
предприниматель операции с недвижимостью  0.000000   
                проведение свадьбы        0.000000   
сотрудник       операции с автомобилем    0.112339   
                операции с недвижимостью  0.086554   
                получение образования     0.110096   
                проведение свадьбы        0.083813   
студент         операции с недвижимостью  0.000000   

                                         Сопоставленная задолженность в %  
income_type     purpose_category                                           
безработный     операции с недвижимостью                              50%  
в декрете       операции с автомобилем                               100%  
госслужащий     операции с автомобилем                                 8%  
                операции с недвижимостью                               5%  
                получение образования                                  8%  
                проведение свадьбы                                     4%  
компаньон       операции с автомобилем                                 8%  
                операции с недвижимостью                               7%  
                получение образования                                  8%  
                проведение свадьбы                                    10%  
пенсионер       операции с автомобилем                                 6%  
                операции с недвижимостью                               5%  
                получение образования                                  7%  
                проведение свадьбы                                     6%  
предприниматель операции с недвижимостью                               0%  
                проведение свадьбы                                     0%  
сотрудник       операции с автомобилем                                11%  
                операции с недвижимостью                               9%  
                получение образования                                 11%  
                проведение свадьбы                                     8%  
студент         операции с недвижимостью                               0%

Заметили, что чаще всего не возвращают кредит в срок те люди, которые имеют профессию `'сотрудник'`. Впредь, был бы с ними более осторожным при выдаче займа.

**ИТОГ:**

* Люди с доходом выше среднего более безопасные клиенты;
* Люди в браке более отвественные плательщики;
* Кол-во детей напрямую отражает выплату в срок кредита;
* Выдача кредита с целью `'операции с недвижимостью'` является более безосапсной в плане задержки срока сдачи;
* Цель кредита `'операции с автомобилем'` является более рискованной;
* Стоит рассматривать источник дохода заемщика, особенно обращая внимание на категорию `'сотрудник'`
